In [1]:
from scraper import WebScraper
from time import sleep
import pandas as pd
from json import dumps
from tqdm import tqdm
from bs4 import BeautifulSoup
from glob import glob

In [2]:
cp = pd.read_excel('./cp_catalogue.xlsx', dtype=str)
cp.drop_duplicates(subset='Código Postal', inplace=True)
cp['Código Postal'] = cp['Código Postal'].str.zfill(5)
already_scraped = glob('./results/*')
already_scraped = [a.split('/')[-1].split('.')[0].split('_')[-1] for a in already_scraped]
cp = cp[~cp['Código Postal'].isin(already_scraped)]

direcciones = [f'{c}, Ciudad de México' for c in cp['Código Postal']]

In [3]:
len(direcciones)

690

In [4]:
source_cates = dict(top_10_semana = "https://www.ubereats.com/mx/search?carid=eyJwbHVnaW4iOiJyZWNvbW1lbmRhdGlvbkZlZWRQbHVnaW4iLCJyZWNvbW1UeXBlIjoidGFnX2Jhc2VkX2Nhcm91c2VsIiwidGFnIjoiaG90dGVzdF90aGlzX3dlZWsifQ%3D%3D&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMlRhaiVDMyVBRG4lMjAyODElMjIlMkMlMjJyZWZlcmVuY2UlMjIlM0ElMjJDaElKOXlVVmdnYl8wWVVSaHFYVDhpWXdtUlklMjIlMkMlMjJyZWZlcmVuY2VUeXBlJTIyJTNBJTIyZ29vZ2xlX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBMTkuMzkxMDY4NiUyQyUyMmxvbmdpdHVkZSUyMiUzQS05OS4xNTM1Mjc3OTk5OTk5OSU3RA%3D%3D&sc=HOME_FEED_ITEM&title=Los%2010%C2%A0m%C3%A1s%20populares%20de%20esta%20semana",

populares_cerca = "https://www.ubereats.com/mx/search?carid=eyJwbHVnaW4iOiJyZWNvbW1lbmRhdGlvbkZlZWRQbHVnaW4iLCJyZWNvbW1UeXBlIjoidG9wX3N0b3Jlc19ieV9jaXR5X3YyIn0%3D&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMlRhaiVDMyVBRG4lMjAyODElMjIlMkMlMjJyZWZlcmVuY2UlMjIlM0ElMjJDaElKOXlVVmdnYl8wWVVSaHFYVDhpWXdtUlklMjIlMkMlMjJyZWZlcmVuY2VUeXBlJTIyJTNBJTIyZ29vZ2xlX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBMTkuMzkxMDY4NiUyQyUyMmxvbmdpdHVkZSUyMiUzQS05OS4xNTM1Mjc3OTk5OTk5OSU3RA%3D%3D&sc=HOME_FEED_ITEM&title=Populares%20cerca%20de%20ti",

locales_populares = "https://www.ubereats.com/mx/search?carid=eyJwbHVnaW4iOiJyZWNvbW1lbmRhdGlvbkZlZWRQbHVnaW4iLCJyZWNvbW1UeXBlIjoidGFnX2Jhc2VkX2Nhcm91c2VsIiwidGFnIjoibW9zdF9wb3B1bGFyX2xvY2FsX3Jlc3RhdXJhbnRzIn0%3D&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMlRhaiVDMyVBRG4lMjAyODElMjIlMkMlMjJyZWZlcmVuY2UlMjIlM0ElMjJDaElKOXlVVmdnYl8wWVVSaHFYVDhpWXdtUlklMjIlMkMlMjJyZWZlcmVuY2VUeXBlJTIyJTNBJTIyZ29vZ2xlX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBMTkuMzkxMDY4NiUyQyUyMmxvbmdpdHVkZSUyMiUzQS05OS4xNTM1Mjc3OTk5OTk5OSU3RA%3D%3D&sc=HOME_FEED_ITEM&title=Restaurantes%20locales%20m%C3%A1s%20populares",

excelentes_precios = "https://www.ubereats.com/mx/search?carid=eyJwbHVnaW4iOiJyZWNvbW1lbmRhdGlvbkZlZWRQbHVnaW4iLCJyZWNvbW1UeXBlIjoidGFnX2Jhc2VkX2Nhcm91c2VsIiwidGFnIjoiZ3JlYXRfbWVhbF9ncmVhdF9wcmljZXMifQ%3D%3D&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMlRhaiVDMyVBRG4lMjAyODElMjIlMkMlMjJyZWZlcmVuY2UlMjIlM0ElMjJDaElKOXlVVmdnYl8wWVVSaHFYVDhpWXdtUlklMjIlMkMlMjJyZWZlcmVuY2VUeXBlJTIyJTNBJTIyZ29vZ2xlX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBMTkuMzkxMDY4NiUyQyUyMmxvbmdpdHVkZSUyMiUzQS05OS4xNTM1Mjc3OTk5OTk5OSU3RA%3D%3D&sc=HOME_FEED_ITEM&title=Buenas%20comidas%20a%20excelentes%20precios")

source_cate = "populares_cerca"

In [5]:
for address in tqdm(direcciones):
    
    scraper = WebScraper()

    scraper.navigate_to_url(source_cates["populares_cerca"])
    
    sleep(1.5)

    # Abrir el menu para gestionar dirección
    scraper.click_element('//*[@id="wrapper"]/header/div/div/div/div/div/div[3]/div[1]/div/a')

    # Entrar al menu para cambiar dirección
    scraper.click_element('//*[@id="wrapper"]/div[3]/div/div/div[2]/div[3]/div/div[2]/div[1]/div[2]/a')

    # Ingresar dirección
    final_address = scraper.enter_address(address, 
                                        input_xpath='//*[@id="location-typeahead-location-manager-input"]', 
                                        selection_xpath='//*[@id="location-typeahead-location-manager-item-0"]'
                                        )
    
    sleep(2)
    cards_data = scraper.scrape_cards('//*[@data-test="feed-desktop"]', 
                                    aditional_info={"cate_source": source_cate,
                                                    "current_address": address,
                                                    "final_address": final_address,
                                                    "platform": 'ubereats'
                                                    }
                                    )
        
    with open(f'./results/ubereats_{source_cate}_{address.split(',')[0]}.json', 'w') as f:
        f.write(dumps(cards_data, indent=4))
    
    soup = BeautifulSoup(scraper.driver.page_source, 'html.parser')
    with open(f'./results/ubereats_{source_cate}_{address.split(',')[0]}.html', 'w') as f:
        f.write(soup.prettify()) 
    
    scraper.close()
    
    sleep(1)

  0%|          | 0/690 [00:00<?, ?it/s]2025-02-01 20:09:56,439 - INFO - Navegando a la URL: https://www.ubereats.com/mx/search?carid=eyJwbHVnaW4iOiJyZWNvbW1lbmRhdGlvbkZlZWRQbHVnaW4iLCJyZWNvbW1UeXBlIjoidG9wX3N0b3Jlc19ieV9jaXR5X3YyIn0%3D&pl=JTdCJTIyYWRkcmVzcyUyMiUzQSUyMlRhaiVDMyVBRG4lMjAyODElMjIlMkMlMjJyZWZlcmVuY2UlMjIlM0ElMjJDaElKOXlVVmdnYl8wWVVSaHFYVDhpWXdtUlklMjIlMkMlMjJyZWZlcmVuY2VUeXBlJTIyJTNBJTIyZ29vZ2xlX3BsYWNlcyUyMiUyQyUyMmxhdGl0dWRlJTIyJTNBMTkuMzkxMDY4NiUyQyUyMmxvbmdpdHVkZSUyMiUzQS05OS4xNTM1Mjc3OTk5OTk5OSU3RA%3D%3D&sc=HOME_FEED_ITEM&title=Populares%20cerca%20de%20ti
2025-02-01 20:10:01,215 - INFO - Elemento en //*[@id="wrapper"]/header/div/div/div/div/div/div[3]/div[1]/div/a clickeado correctamente.
2025-02-01 20:10:02,384 - INFO - Elemento en //*[@id="wrapper"]/div[3]/div/div/div[2]/div[3]/div/div[2]/div[1]/div[2]/a clickeado correctamente.
2025-02-01 20:10:06,487 - INFO - Elemento en //*[@id="location-typeahead-location-manager-item-0"] clickeado correctamente.
2025-02-01 20:1